In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)


# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:12:13 of 0:12:14 remaining]
Chain 1:  10% [0:01:04 of 0:01:11 remaining]
Chain 1:  20% [0:01:00 of 0:01:15 remaining]
Chain 1:  30% [0:00:53 of 0:01:16 remaining]
Chain 1:  40% [0:00:45 of 0:01:16 remaining]
Chain 1:  50% [0:00:37 of 0:01:15 remaining]
Chain 1:  60% [0:00:30 of 0:01:15 remaining]
Chain 1:  70% [0:00:23 of 0:01:15 remaining]
Chain 1:  80% [0:00:15 of 0:01:15 remaining]
Chain 1:  90% [0:00:07 of 0:01:14 remaining]
Chain 1: 100% [0:00:00 of 0:01:14 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:43 of 0:00:48 remaining]
Chain 2:  20% [0:00:50 of 0:01:02 remaining]
Chain 2:  30% [0:00:48 of 0:01:08 remaining]
Chain 2:  40% [0:00:42 of 0:01:10 remaining]
Chain 2:  50% [0:00:35 of 0:01:11 remaining]
Chain 2:  60% [0:00:28 of 0:01:11 remaining]
Chain 2:  70% [0:00:22 of 0:01:12 remaining]
Chain 2:  80% [0:00:14 of 0:01:12 remaining]
Chain 2:  90% [0:00:07 of 0:01:12 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[24.7635 113.324 0.923724; 28.4582 113.326 0.908273; … ; 25.9695 113.538 0.91149; 26.7477 112.384 0.93256]

[30.494 117.833 0.816861; 30.95 116.717 0.832155; … ; 24.7071 113.187 0.923344; 24.7167 113.18 0.908643]

[25.4349 113.423 0.913134; 26.2802 113.886 0.889451; … ; 24.8781 112.593 0.925211; 25.9084 112.593 0.925211]

Show draws summary

In [3]:
describe(sim)

##-

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean        SD        Naive SE       MCSE        ESS   
     s2  26.1733350 1.99427199 0.01213675283 0.0195193085 9000.0000
beta[1] 113.8805165 1.89983567 0.01156203168 0.0338775178 3144.9150
beta[2]   0.9050092 0.04185932 0.00025474771 0.0007407521 3193.2917

Quantiles:
           2.5%        25.0%       50.0%       75.0%        97.5%   
     s2  22.561815  24.78285800  26.0736587  27.4423801  30.42276217
beta[1] 110.129354 112.58086585 113.8893282 115.1676026 117.59213259
beta[2]   0.823327   0.87660118   0.9049346   0.9336288   0.98694646



*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*